In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData

In [63]:
def getAllFeatureData():
    matfiles = glob.glob('Preprocessed_EEG/*.mat')
    matfiles.sort()
    labels=loadmat(matfiles[15])
    label_list=np.ndarray.tolist(labels['label'])[0]
    x=getRawArrayData()
    sliced_epochs={}
    combined = np.zeros((1,311))
    selected_funcs = ['hjorth_complexity','hjorth_mobility',('diffEnt', compute_diffEnt),'std','skewness']
    for subs in x:
        for clip in x[subs]:
            sliced_epochs[clip]= mne.make_fixed_length_epochs(x[0][clip], duration=1, preload=True,verbose=0)
        for cut in sliced_epochs:
            label_used=label_list[cut]
            epoch_array=mne.Epochs.get_data(sliced_epochs[cut])
            extracted_data=extract_features(epoch_array,200,selected_funcs)
            label_col = np.zeros((extracted_data.shape[0],1)) + label_used
            oneCom = np.hstack((extracted_data,label_col))
            combined = np.vstack((combined,oneCom))
    combined = np.delete(combined, 0, axis=0)
    return combined

In [64]:
def compute_diffEnt(data):
    return scipy.stats.differential_entropy(data, axis=-1)